# Tone Generation: IT-Motor Split Range

In [ ]:
import numpy as np
import librosa as lba
import soundfile as sf
from random import random

In [ ]:
# Set path to stimulus folder
stimuli_folder = '../stimuli/'

# Set sampling rate (Wave Shield can only play 22 kHz)
sr = 22050

# Set up duration of section of the amplitude envelope 
tone_duration = 250
rise_duration = 10
perc_duration = 240
fade_duration = 10
rise_length = int(rise_duration * sr / 1000)
perc_length = int(perc_duration * sr / 1000)
fade_length = int(fade_duration * sr / 1000)

# Calculate frequencies for all tones from C2 to B7 (A4 = 440 Hz) by converting piano key numbers to Hz
# (See en.wikipedia.org/wiki/Piano_key_frequencies for formula)
piano_keynums = np.arange(16, 88)
freqs = 440 * 2 ** ((piano_keynums - 49) / 12)

# Generate names of all pitches from C2 to B7
pitch_classes = ('C', 'Cs', 'D', 'Ds', 'E', 'F', 'Fs', 'G', 'Gs', 'A', 'As', 'B')
pitch_names = [c + str(o) for o in range(2, 8) for c in pitch_classes]

# Map pitch names to their frequencies
tones = {p: freqs[i] for i, p in enumerate(pitch_names)}

# Generate Tones

In [ ]:
for pitch in ('A2', 'Ds3', 'A3', 'Ds4', 'A4', 'Ds5', 'A5', 'Ds6', 'A6', 'Ds7', 'A7'):

    f = tones[pitch]
    
    # Generate fundamental frequency (co)sine wave
    tone = lba.tone(f, sr=sr, duration=tone_duration / 1000)
    
    # Add first three harmonics with slope of -6 db/half amplitude per octave
    for i in range(2):
        phase = random() * 2 * np.pi
        tone += lba.tone(f * (i + 2), sr=sr, duration=tone_duration / 1000, phi=phase) / (i + 2)
    
    # Rescale waveform to range [-1, 1] to prevent clipping
    tone /= np.abs(tone).max()

    # Apply exponential fade to create percussive envelope
    tone[-perc_length:] *= np.geomspace(1, .01, perc_length)
    # Apply short linear fade to ending so that amplitude fades to 0
    tone[-fade_length:] *= np.linspace(1, 0, fade_length)
    # Apply sharp linear rise to start of tone
    tone[:rise_length] *= np.linspace(0, 1, rise_length)
    
    # Save tone
    sf.write('../stimuli/raw/tone%s.wav' % pitch, tone, sr)

## (Then balance tones in Audacity)